In [1]:
import praw
import pandas as pd
from praw.models import MoreComments

In [2]:
#reddit user log in
reddit_read_only = praw.Reddit(client_id = 'JJ_URJnuIU9HvdINJ-17LQ',
                               client_secret = 'WBd1WSOpJVnefOx2ZjIgbeA6OCfbzw',
                               user_agent = 'appy_apperson')

In [3]:
#calling the subreddit we want to scrape from
subreddit = reddit_read_only.subreddit('cuboulder')

In [4]:
#getting the info for the top posts of all time
#it seems reddit has a limit of 1000 queries.
posts = subreddit.top('all',limit=1000)

#create a dictionary to store the data
posts_dict = {'title':[], 'date':[],'Post Text':[], 'ID':[],
              'score':[], 'total comments':[], 'post url':[]}

#loops through and adds the posts to the dictionary
for post in posts:
    posts_dict['title'].append(post.title)
    posts_dict['date'].append(post.created_utc)
    posts_dict['Post Text'].append(post.selftext)
    posts_dict['ID'].append(post.id)
    posts_dict['score'].append(post.score)
    posts_dict['total comments'].append(post.num_comments)
    posts_dict['post url'].append(post.url)

/var/folders/qr/_sqyrs1n54zc6_vg5wwvsdqr0000gn/T/ipykernel_35051/1510095379.py:3: DeprecationWarning: Positional arguments for 'BaseListingMixin.top' will no longer be supported in PRAW 8.
Call this function with 'time_filter' as a keyword argument.
  posts = subreddit.top('all',limit=1000)


In [5]:
#creates a DF to store
top_posts = pd.DataFrame(posts_dict)
#changes the time stamp from epoch to gmt
top_posts['date'] = pd.to_datetime(top_posts['date'], unit='s')
top_posts.head()

,title,date,Post Text,ID,score,total comments,post url
0,Drone shots of our desolate campus during the ...,2020-08-16 14:36:45,,iate23,875,71,https://v.redd.it/selcvau5kdh51
1,Buff Bus Good - CU's busses were used to help ...,2021-03-23 00:52:44,,mb2m07,724,17,https://i.redd.it/dfxsfufydoo61.png
2,this sub is so negative all the time so here’s...,2020-09-09 00:24:48,,ip5lcr,695,17,https://i.redd.it/07d0706in0m51.jpg
3,"And to the rest of you, a pleasant evening",2021-03-07 06:01:24,,lzkigx,667,82,https://i.redd.it/mptmf05oqjl61.jpg
4,Streaming costs,2020-09-23 00:34:39,,ixzx6g,627,18,https://i.redd.it/tew68s81mso51.jpg


In [6]:
print(top_posts.shape)
print(top_posts.date.max())
print(top_posts.date.min())

(999, 7)
2023-03-08 18:54:25
2016-09-02 16:37:41


In [7]:
#filtered the results to only things since 2020
_2020 = top_posts.loc[top_posts.date > '2020-01-01']
#reset the index 
_2020.reset_index(inplace=True)
_2020.shape

(799, 8)

In [ ]:
#create a dictionary to store results
comments = {'ID':[], 'title':[], 'date':[],'comment':[]}

#run a loop through our 2020 posts by post id. 
for num in range(len(_2020)):
    submission = reddit_read_only.submission(_2020['ID'][num])
    submission.comments.replace_more(limit=None)
    for comment in submission.comments.list():
        #print(comment.body)
        comments['ID'].append(_2020['ID'][num])
        comments['title'].append(_2020['title'][num])
        comments['date'].append(_2020['date'][num])
        comments['comment'].append(comment.body)
    print(f'Progress: {num+1} out of {len(_2020)} complete.', end='\r' )

In [ ]:
#turn into a dataframe
cu_comments = pd.DataFrame(comments)
cu_comments.shape

In [ ]:
cu_comments.tail()

In [ ]:
#write our results to a csv
cu_comments.to_csv('cu_comments.csv')